In [1]:
#We are solving the question by Linear programming in python of operational research technique



In [2]:
#Importing all the necessary libraries

from pulp import *
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#We will use the pulp library of python for linear programming

In [4]:
#Sets of the sources and destination .

SOURCES = ['Chennai', 'Thiruvallur']
DESTINATIONS = ['PlantA', 'PlantB', 'PlantC']

In [5]:
#Dictionary of number of buses in Demand (DEMAND)
#these numbers are calculated by the percentage of employees given in the information ,
#these numbers represent the number of buses the plant requires
demand = {  
    'PlantA' : 30,                   
    'PlantB': 33,
    'PlantC' : 7
}
#As the supply (i.e the number of buses the stations chennai and thiruvallur can accomodate )  was given to be 60 and 37 respectively
# in normal days,but in the pandemic situation as the demand of buses was droped the supply also needs to be decreased.
# So for making the problem  BALANCED , we take the supply as variable.


In [6]:
x = LpVariable(name="x", lowBound=0 , upBound=60 , cat = 'Integer') 
#Number of buses chennai station would accomodate with max. limit of 60                                        
y = LpVariable(name="y", lowBound=0 , upBound=37 , cat = 'Integer') 
#Number of buses thiruvallur station would accomodate with max. limit of 37       

In [7]:
#Dictionary of number of buses in Supply (SUPPLY)
supply = {
    'Chennai' : x  ,
    'Thiruvallur' : y
}

In [8]:
#Dictionary of costs of demand and supply of buses
cost = {
    'Chennai' : { 'PlantA' : 50, 'PlantB' : 55, 'PlantC' : 32},      #cost from chennai station to respective plants
    'Thiruvallur': { 'PlantA' : 30, 'PlantB' : 25, 'PlantC' : 55}    #cost from thiruvallur station to respective plants
}
#here the cost is in mutiple of 100...scale 1 = 100

In [9]:
#The whole steps above can be skipped if the data was large and in excel formal. Using pandas and numpy libraries for formating and reading excel can be done in single step

#df = pd.read_excel('file_name.extension')


In [10]:
#Setting problem variables (DECISION VARIABLES)

#initializning the model
prob = LpProblem("Transport",LpMinimize)   
#Defining the problem model(prob) with name as Transport and defining the function to find the minimum

routes = [(i,j) for i in SOURCES for j in DESTINATIONS]  
#defining the variables routes for Cij (Number of bus in matrix with ith station and jth plant)

#the decision making variables (number of busses) with number of buses can be  zero for some routes
amount_vars = LpVariable.dicts('No._Bus',(SOURCES,DESTINATIONS),lowBound=0 , cat= 'Integer') 
#reading the dictionary and then initializing the Cij


In [11]:
#Now telling the model for operations (adding the conditions)
prob += lpSum(amount_vars[i][j]*cost[i][j] for (i,j) in routes)  #now multiplying the Cij(number of buses) with the cost

In [12]:
for j in DESTINATIONS:
    prob += lpSum((amount_vars[i][j]) for i in SOURCES) == demand[j]
    #total number of buses to each plant must be equal to the demand 
for i in SOURCES:
    prob += lpSum((amount_vars[i][j]) for j in DESTINATIONS) <= supply[i]   
    #total number of buses from each station should be less then or equal to the supply 

In [13]:
prob += lpSum([x, y]) == 70       #the total number of buses from the stations must be equal to the demand (70)

In [14]:
prob.solve()          
print('Status:',LpStatus[prob.status])     #priting the status of the solution

Status: Optimal


In [15]:
for v in prob.variables():
        print(v.name, '=' ,v.varValue)

No._Bus_Chennai_PlantA = 26.0
No._Bus_Chennai_PlantB = 0.0
No._Bus_Chennai_PlantC = 7.0
No._Bus_Thiruvallur_PlantA = 4.0
No._Bus_Thiruvallur_PlantB = 33.0
No._Bus_Thiruvallur_PlantC = 0.0
x = 33.0
y = 37.0


In [16]:
#So the number of buses from station Chennai (x) is 33
#So the number of buses from station thiruvallur (y) is 37

In [17]:
print('Total_cost_of_transportation = ', value(prob.objective)*100)

Total_cost_of_transportation =  246900.0


In [ ]:
#graphs can be drawn using the library matplotlib for better visualization